In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.4 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
import random
import json
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from sklearn.metrics import classification_report, fbeta_score
from timeit import default_timer as timer
from scipy.optimize import basinhopping
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

In [4]:
# encode classes using multi label Binarizer
def get_classes_list():
    classes = ['råvaremarked', 'aksjer', 'børs', 'valuta', 'other']
    return classes, len(classes)
classes, num_classes = get_classes_list()
mlb = MultiLabelBinarizer(classes=classes)

In [5]:
tokenizer = BertTokenizer.from_pretrained('NbAiLab/nb-bert-base', do_lower_case=True)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/746 [00:00<?, ?B/s]

In [6]:
df = pd.read_csv('/content/drive/MyDrive/data.csv')

In [7]:
def label_extract(row):
  cats = []
  if type(row['category']) is str:
    # print(type(row['category']))
    cat_list = eval(row['category'])
    for i in cat_list:
      cats.append(i['name'])
  else:
    cats.append('other')

  return cats

df['category'] = df.apply(label_extract, axis = 1)

In [8]:
X = df['full_text_with_title']

In [9]:
encoded_data = tokenizer.batch_encode_plus(
    X, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest', 
    truncation=True,
    max_length=512, 
    return_tensors='pt'
)

In [11]:
input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']

In [12]:
dataset = TensorDataset(input_ids, attention_masks)

In [13]:
batch_size = 8

dataloader = DataLoader(dataset, 
                              sampler=RandomSampler(dataset), 
                              batch_size=batch_size)

In [16]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("NbAiLab/nb-bert-base", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=5,
                                                           )

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at NbAiLab/nb-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initi

In [17]:
device = torch.device('cuda:0')
checkpoint = torch.load('/content/drive/MyDrive/finetune1-model_best.pth', map_location=device)
model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])
threshold = checkpoint['threshold']
threshold = torch.from_numpy(threshold)

In [18]:
model.eval()
model.to(device)

loss_val_total = 0
predictions = []

for batch in dataloader:
    
    batch = tuple(b.to(device) for b in batch)
    
    inputs = {'input_ids':      batch[0],
              'attention_mask': batch[1],
              }
    
    with torch.no_grad():        
        outputs = model(**inputs)
        
    logits = outputs['logits']
    pred = torch.sigmoid(logits)

    pred_labels = torch.as_tensor((pred.cpu() - threshold) > 0, dtype=torch.int32)

    predictions.append(pred_labels.data.cpu().numpy())


predictions = np.vstack(predictions)

In [19]:
predictions.shape

(1867, 5)

In [25]:
predictions

array([[0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       ...,
       [1, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1]], dtype=int32)

In [31]:
preds = []
for i in range(predictions.shape[0]):
    pred = []
    if predictions[i,0] == 1:
        pred.append('råvaremarked')
    if predictions[i,1] == 1:
        pred.append('aksjer')
    if predictions[i,2] == 1:
        pred.append('børs')
    if predictions[i,3] == 1:
        pred.append('valuta')
    if predictions[i,4] == 1:
        pred.append('other')
    preds.append(pred)


In [33]:
df['predictions'] = preds

In [23]:
del df['Unnamed: 0']

In [34]:
df.head()

,_id,full_text_with_title,category,råvaremarked,aksjer,børs,valuta,other,predictions
0,62eb8f5b148b7abcd515bcf4,På bare noen uker ble et «lite» Hongkong-baser...,[børs],0,0,1,0,0,[børs]
1,62eb8f92148b7abcd515c662,Usikkerhet preger aksjemarkedet: – Det er en ø...,[børs],0,0,1,0,0,[other]
2,62ebc71d148b7abcd51f3313,Bloomberg: Schibsted forbereder salg av Lendo ...,[børs],0,0,1,0,0,[børs]
3,62ebcb6d148b7abcd51ff3cb,Wall Street åpner flatt Oppturen fra onsdag sm...,[børs],0,0,1,0,0,[råvaremarked]
4,62f0b7b9148b7abcd50013ed,Oslo Børs stiger fra start På ukens første han...,"[aksjer, børs]",0,1,1,0,0,[råvaremarked]


In [35]:
i = 0
for index, row in df.iterrows():
    print(row['category'], row['predictions'])
    if row['category']==row['predictions']:
        i+=1
print(i)

['børs'] ['børs']
['børs'] ['other']
['børs'] ['børs']
['børs'] ['råvaremarked']
['aksjer', 'børs'] ['råvaremarked']
['aksjer', 'børs'] ['børs']
['aksjer', 'børs'] ['børs']
['aksjer', 'børs'] ['råvaremarked']
['aksjer', 'børs'] ['børs']
['aksjer', 'børs'] ['børs']
['aksjer', 'børs'] ['other']
['aksjer', 'børs'] ['other']
['aksjer', 'børs'] ['aksjer', 'børs']
['aksjer', 'børs'] ['børs']
['børs'] ['other']
['aksjer'] ['råvaremarked']
['råvaremarked'] ['børs']
['aksjer', 'børs'] ['other']
['børs'] ['råvaremarked']
['aksjer', 'børs'] ['other']
['børs'] ['aksjer', 'børs']
['råvaremarked'] ['børs']
['børs'] ['børs']
['børs'] ['other']
['aksjer', 'børs'] ['other']
['børs'] ['other']
['råvaremarked'] ['børs']
['aksjer', 'børs'] ['other']
['aksjer', 'børs'] ['råvaremarked']
['børs'] ['råvaremarked', 'aksjer', 'børs']
['børs'] ['råvaremarked']
['aksjer', 'børs'] ['råvaremarked']
['børs'] ['børs']
['børs'] ['aksjer', 'børs']
['børs'] ['råvaremarked']
['børs'] ['råvaremarked', 'aksjer', 'børs']
['

In [36]:
df.to_csv('data_with_preds.csv')

In [37]:
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

In [38]:
test_df.head()

,Unnamed: 0,_id,article_tags,datetime,extracted_tags,full_text,title,topics,source,full_text_with_title
0,0,62daa47b79f22232e10b1497,"[""Teknologi"",""metaverse"",""Nyheter""]",2022-01-14T00:00:00.000Z,"[""milllioner"",""kjølvannet"",""tilfeldigvis"",""bol...",Siden Facebook i fjor kunngjorde et navneskift...,Eiendomsprisene er opp 500 prosent i «metaverset»,"[""Metaverse"",""Stykk"",""Boligpris"",""Teknologi"",""...",finansavisen.no,Eiendomsprisene prosent metavers Facebook kunn...
1,1,62daa4c779f22232e10b2376,"[""Nyheter"",""Makro""]",2022-07-20T06:10:05.000Z,"[""husholdningene"",""energiprisøkning"",""styrings...",Konsumprisindeksen er nå oppe på det høyeste n...,Høyeste britisk inflasjon på 40 år,"[""Lønnsøkning"",""Måned"",""Styringsrente"",""Hushol...",finansavisen.no,Høyeste britisk inflasjon konsumprisindeks opp...
2,2,62de880579f22232e1a89005,"[""breaking"",""federal reserve"",""jerome powell"",...",2022-06-15T18:00:56.000Z,"[""styringsrenten"",""rentehevingene"",""inflasjons...",Federal Reserve hever styringsrenten med 75 ba...,Fed med monsterheving,"[""Rentebeslutning"",""federal reserve"",""Styrings...",finansavisen.no,Fed monsterheving Federal Reserve heve styring...
3,3,62deb2e979f22232e1aff02b,"[""porsche"",""vw"",""Bil"",""Nyheter"",""Børs"",""børsno...",2022-07-25T00:00:00.000Z,"[""aksjemarkedet"",""påtroppende"",""elbiler"",""indu...",De siste årene har vært preget av store utford...,"Hva nå, Volkswagen?","[""Industrigigant"",""Børsnotering"",""Tesla"",""Kons...",finansavisen.no,hva Volkswagen år prege utfordring tysk Volksw...
4,4,62dfb08d79f22232e1d674e5,"[""WEST"",""Nyheter"",""Shipping"",""Børs""]",2022-06-14T06:47:40.000Z,"[""rekordhøyt"",""halvår"",""aksjen"",""rateoppgang"",...",Western Bulk håver inn penger på børsuroen. Nå...,Tidenes resultat for Western Bulk,"[""Aksje"",""Tørrlastrederi"",""Rateoppgang"",""Marke...",finansavisen.no,Tidene resultat Western Bulk Western Bulk håve...


In [39]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1408 entries, 0 to 1407
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            1408 non-null   int64 
 1   _id                   1408 non-null   object
 2   article_tags          1408 non-null   object
 3   datetime              1408 non-null   object
 4   extracted_tags        1209 non-null   object
 5   full_text             1408 non-null   object
 6   title                 1408 non-null   object
 7   topics                1408 non-null   object
 8   source                1408 non-null   object
 9   full_text_with_title  1408 non-null   object
dtypes: int64(1), object(9)
memory usage: 110.1+ KB


In [40]:
x = test_df['full_text_with_title']

In [41]:
encoded_data_x = tokenizer.batch_encode_plus(
    x, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest', 
    truncation=True,
    max_length=512, 
    return_tensors='pt'
)

In [42]:
input_ids_x = encoded_data_x['input_ids']
attention_masks_x = encoded_data_x['attention_mask']

dataset_x = TensorDataset(input_ids_x, attention_masks_x)

In [43]:
dataloader_x = DataLoader(dataset_x, 
                              sampler=RandomSampler(dataset_x), 
                              batch_size=8)

In [14]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("NbAiLab/nb-bert-base", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=5,
                                                           )

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at NbAiLab/nb-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initi

In [17]:
device

device(type='cuda', index=0)

In [15]:
device = torch.device('cuda:0')
checkpoint = torch.load('/content/drive/MyDrive/finetune1-model_best.pth', map_location=device)
model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])
threshold = checkpoint['threshold']
threshold = torch.from_numpy(threshold)

In [44]:
model.eval()
model.to(device)

loss_val_total = 0
predictions, true_vals, val_his, mAP = [], [], [], []
print("Starting Validation")
for batch in dataloader_x:
    
    batch = tuple(b.to(device) for b in batch)
    
    inputs = {'input_ids':      batch[0],
              'attention_mask': batch[1],
              # 'labels':         batch[2],
              }
    
    # labels = batch[2]
    with torch.no_grad():        
        outputs = model(**inputs)
        
    logits = outputs['logits']
    pred = torch.sigmoid(logits)

    pred_labels = torch.as_tensor((pred.cpu() - threshold) > 0, dtype=torch.int32)

    predictions.append(pred_labels.data.cpu().numpy())
    # true_vals.append(label_ids)
    

# loss_val_avg = loss_val_total/len(dataloader_x) 
# val_his.append(loss_val_avg)

predictions = np.vstack(predictions)

Starting Validation


In [45]:
predictions.shape


(1408, 5)

In [48]:
preds = []
for i in range(predictions.shape[0]):
    pred = []
    if predictions[i,0] == 1:
        pred.append('råvaremarked')
    if predictions[i,1] == 1:
        pred.append('aksjer')
    if predictions[i,2] == 1:
        pred.append('børs')
    if predictions[i,3] == 1:
        pred.append('valuta')
    if predictions[i,4] == 1:
        pred.append('other')
    preds.append(pred)

In [49]:
del test_df['Unnamed: 0']

In [50]:
test_df['predictions'] = preds

In [51]:
test_df

,_id,article_tags,datetime,extracted_tags,full_text,title,topics,source,full_text_with_title,predictions
0,62daa47b79f22232e10b1497,"[""Teknologi"",""metaverse"",""Nyheter""]",2022-01-14T00:00:00.000Z,"[""milllioner"",""kjølvannet"",""tilfeldigvis"",""bol...",Siden Facebook i fjor kunngjorde et navneskift...,Eiendomsprisene er opp 500 prosent i «metaverset»,"[""Metaverse"",""Stykk"",""Boligpris"",""Teknologi"",""...",finansavisen.no,Eiendomsprisene prosent metavers Facebook kunn...,[other]
1,62daa4c779f22232e10b2376,"[""Nyheter"",""Makro""]",2022-07-20T06:10:05.000Z,"[""husholdningene"",""energiprisøkning"",""styrings...",Konsumprisindeksen er nå oppe på det høyeste n...,Høyeste britisk inflasjon på 40 år,"[""Lønnsøkning"",""Måned"",""Styringsrente"",""Hushol...",finansavisen.no,Høyeste britisk inflasjon konsumprisindeks opp...,[other]
2,62de880579f22232e1a89005,"[""breaking"",""federal reserve"",""jerome powell"",...",2022-06-15T18:00:56.000Z,"[""styringsrenten"",""rentehevingene"",""inflasjons...",Federal Reserve hever styringsrenten med 75 ba...,Fed med monsterheving,"[""Rentebeslutning"",""federal reserve"",""Styrings...",finansavisen.no,Fed monsterheving Federal Reserve heve styring...,[other]
3,62deb2e979f22232e1aff02b,"[""porsche"",""vw"",""Bil"",""Nyheter"",""Børs"",""børsno...",2022-07-25T00:00:00.000Z,"[""aksjemarkedet"",""påtroppende"",""elbiler"",""indu...",De siste årene har vært preget av store utford...,"Hva nå, Volkswagen?","[""Industrigigant"",""Børsnotering"",""Tesla"",""Kons...",finansavisen.no,hva Volkswagen år prege utfordring tysk Volksw...,[other]
4,62dfb08d79f22232e1d674e5,"[""WEST"",""Nyheter"",""Shipping"",""Børs""]",2022-06-14T06:47:40.000Z,"[""rekordhøyt"",""halvår"",""aksjen"",""rateoppgang"",...",Western Bulk håver inn penger på børsuroen. Nå...,Tidenes resultat for Western Bulk,"[""Aksje"",""Tørrlastrederi"",""Rateoppgang"",""Marke...",finansavisen.no,Tidene resultat Western Bulk Western Bulk håve...,"[aksjer, børs, other]"
...,...,...,...,...,...,...,...,...,...,...
1403,632850b99b10daf919af7c21,"[""baseball"",""samlerobjekt"",""Nyheter"",""Sport"",""...",2022-08-29T12:42:56.000Z,NaN,Et baseballkort fra 1952 knuste Diego Maradona...,Baseballkort solgt for 125 mill.,"[""Baseball"",""Argentiner"",""Samlerobjekt"",""Håndf...",finansavisen.no,baseballkort selge mill. baseballkort knuse Di...,[other]
1404,632850bf9b10daf919af7d22,"[""maradona"",""Nyheter"",""Sport"",""Børs""]",2022-05-04T16:52:15.000Z,NaN,Diego Maradonas drakt fra den legendariske VM-...,Maradonas «Guds hånd»-drakt solgt for over 80 ...,"[""Spillertunnel"",""Pund"",""Fotballdrakte"",""Tid""]",finansavisen.no,Maradona Gud hånd»-drakt selge million Diego M...,"[aksjer, børs]"
1405,632850c29b10daf919af7d84,"[""strømstøtte"",""virke"",""Nyheter"",""Politikk""]",2022-09-16T08:43:05.000Z,NaN,Virke er fornøyd med regjeringens strømstøtteo...,Virke: Vil forkorte strømkrisen og redde arbei...,"[""Strømstøtteordning"",""Samfunnspåvirkning"",""Næ...",finansavisen.no,virke forkorte strømkrise redd arbeidsplass Vi...,[other]
1406,632850c59b10daf919af7e58,"[""NHO"",""strømstøtte"",""Nyheter"",""Politikk"",""Ene...",2022-09-16T08:37:53.000Z,NaN,NHO er glad for regjeringens ordning for strøm...,NHO om strømstøtten: Større bedrifter må få mer,"[""Politikk"",""Strømpakk""]",finansavisen.no,NHO strømstøtt Større bedrift NHO glad regjeri...,"[aksjer, børs, other]"


In [52]:
test_df.to_csv('test_with_preds.csv')

In [18]:
dataset = pd.DataFrame({'råvaremarked': p[:, 0], 
                        'aksjer': p[:, 1],
                        'børs': p[:, 2],
                        'valuta': p[:, 3],
                        'other': p[:, 4]}
                       )

In [19]:
dataset['article_tags'] = test_df['article_tags']
dataset['_id'] = test_df['_id']

In [20]:
dataset.head()

,råvaremarked,aksjer,børs,valuta,other,article_tags,_id
0,1,0,0,0,1,"[""Teknologi"",""metaverse"",""Nyheter""]",62daa47b79f22232e10b1497
1,0,0,0,0,1,"[""Nyheter"",""Makro""]",62daa4c779f22232e10b2376
2,0,0,0,0,1,"[""breaking"",""federal reserve"",""jerome powell"",...",62de880579f22232e1a89005
3,0,0,0,0,1,"[""porsche"",""vw"",""Bil"",""Nyheter"",""Børs"",""børsno...",62deb2e979f22232e1aff02b
4,1,0,0,0,0,"[""WEST"",""Nyheter"",""Shipping"",""Børs""]",62dfb08d79f22232e1d674e5


In [21]:
råvaremarked_keyword = ['stål', 'oljeprodusenter', 'oljelagertall', 'brentolje', 'oljebransjen', 'skiferolje', 'olje/energi', 'litium', 'norsk olje og gass', 
                        'drivstoff', 'oljelager', 'drivstoffpriser', 'biodrivstoff', 'oljeindustrien', 'oljemarkedet', 'oljepolitikk', 'aluminium', 'oljeselskaper', 
                        'oljelisenser', 'gull', 'olje', 'oljeprisutslagene', 'olje og gass', 'bensinprisen', 'olje & gass', 'oljeetterspørsel', 'oljedirektoratet', 
                        'brent', 'bensinpris', 'drivstoffprisene', 'oljeservice', 'amerikansk lettolje (wti)', 'oljeleting', 'wti-olje', 'oljeeksport', 'drivstoffappen', 
                        'energi og råvarer', 'oljeproduksjon', 'oljeskatt', 'oljekraftverk', 'stopp oljeletinga', 'oljefunn', 'norsk stål', 'oljefondets milliardkjøp', 
                        'stålproduksjon', '«oljeskatt» på oppdrett', 'oljearbeidere', 'bensin', 'bensinkrig', 'kull', 'oljeutvinning', 'råvarer', 'nikkel', 'brent-olje']
aksjer_keyword = ['aksjeanbefaling', 'dn aksjer', 'aksjelive', 'aksjemarkedet', 'aksjetipset', 'meme-aksje', 'aksjefond', 'it-aksjer', 'aksjeposisjoner', 
                  'aksje-nm 2022', 'aksjesparing', 'aksje-nm', 'aksjesparekonto', 'aksjepraten', 'kjøpsanbefaling', 'aksjeanalyse', 'lakseaksjer', 'anbefaling', 
                  'aksjehandel', 'aksjetips', 'oljeaksjer', 'aksje', 'teknologiaksjer', 'aksjer', 'investorenes aksjekjøp', 'flyaksjer', 'vekstaksjer', 'aksjeselskaper']
børs_keyword = ['asia-børsene', 'børs', 'børsdagen', 'børsen', 'børsene i asia', 'børsfall', 'børsintervju', 'børskommentar', 'børsmorgen', 'børsnotering', 'børsnoteringer', 
                'børsoppdatering', 'børsåpning', 'dagligvarebørsen', 'dette vil påvirke oslo børs', 'etterbørs', 'helsinki-børsen', 'hongkong-børsen', 'kryptovalutabørs', 
                'new york-børsen', 'nytt fra børsselskapene', 'oslo børs', 'oslo børs seafood index (obsfx)', 'seoul-børsen', 'shanghai-børsen', 'singapore-børsen', 'stockholmsbørsen', 
                'sydney-børsen', 'tokyo-børsen']
valuta_keyword = ['kryptoutvinning', 'krypto', 'kryptovalutabørs', 'valutamarkedet', 'kronekurs', 'dollar', 'kryptokunst', 'kryptoboomen', 'blokkjede', 'kryptovaluta', 'kronekursen', 
                  'norsk krone', 'kronen', 'lokal valuta', 'kryptovault', 'blokkjedeteknologi', 'nft', 'euro', 'krone', 'valuta', 'digital valuta']

In [22]:
def category_assignment(row, råvaremarked_keyword=råvaremarked_keyword, aksjer_keyword=aksjer_keyword, 
                        børs_keyword=børs_keyword, valuta_keyword=valuta_keyword):
    """input keyword list and return corresponding category"""
    if type(row['article_tags']) is str:
        article_tags = json.loads(row['article_tags'])
        article_tags = [item.lower() for item in article_tags]
        article_tags_set = set(article_tags)
        print(article_tags_set)
        category = []
        #check if article belongs to råvaremarked_keyword
        råvaremarked_keyword_set = set(råvaremarked_keyword)
        checker_råvaremarked = råvaremarked_keyword_set.intersection(article_tags_set)

        #check if article belongs to aksjer_keyword
        aksjer_keyword_set = set(aksjer_keyword)
        checker_aksjer = aksjer_keyword_set.intersection(article_tags_set)

        #check if article belongs to børs_keyword
        børs_keyword_set = set(børs_keyword)
        checker_børs = børs_keyword_set.intersection(article_tags_set)

        #check if article belongs to valuta_keyword
        valuta_keyword_set = set(valuta_keyword)
        checker_valuta = valuta_keyword_set.intersection(article_tags_set)
        if checker_råvaremarked:
            category.append('råvaremarked')

        if checker_aksjer:
            category.append('aksjer')

        if checker_børs:
            category.append('børs')

        if checker_valuta:
            category.append('valuta')
        
        if category:
            return category
        else:
            return ['other']

In [23]:
dataset['category'] = dataset.apply(category_assignment, axis = 1)

{'nyheter', 'teknologi', 'metaverse'}
{'nyheter', 'makro'}
{'danske bank', 'renteheving', 'federal reserve', 'breaking', 'frank jullum', 'inflasjon', 'nyheter', 'knut a magnussen', 'dnb markets', 'jerome powell', 'makro', 'markedsuro'}
{'porsche', 'børsnotering', 'vw', 'bil', 'nyheter', 'tesla', 'børs'}
{'shipping', 'west', 'børs', 'nyheter'}
{'kvartalsrapport', 'energi', 'breaking', 'nyheter', 'børs', 'equinor', 'eqnr'}
{'bank', 'nyheter', 'finans', 'credit suisse', 'sveits'}
{'greensill capital', 'credit suisse', 'archegos capital', 'nyheter', 'hedgefond', 'børs', 'finans'}
{'oanda', 'wall street', 'credit suisse', 'archegos capital', 'baidu', 'nyheter', 'marginskvis', 'goldman sachs', 'børs', 'hedgefond', 'resultatvarsel', 'discovery', 'nomura', 'viacomcbs', 'finans'}
{'wall street', 'dow jones', 's&p', 'børsoppdatering', 'nyheter', 'nasdaq', 'børs', 's&p500'}
{'spirit airlines', 'easyjet', 'norse atlantic airways', 'nyheter', 'norse atlantic', 'luftfart', 'norwegian'}
{'otec', 'nyh

In [24]:
dataset.head()

,råvaremarked,aksjer,børs,valuta,other,article_tags,_id,category
0,1,0,0,0,1,"[""Teknologi"",""metaverse"",""Nyheter""]",62daa47b79f22232e10b1497,[other]
1,0,0,0,0,1,"[""Nyheter"",""Makro""]",62daa4c779f22232e10b2376,[other]
2,0,0,0,0,1,"[""breaking"",""federal reserve"",""jerome powell"",...",62de880579f22232e1a89005,[other]
3,0,0,0,0,1,"[""porsche"",""vw"",""Bil"",""Nyheter"",""Børs"",""børsno...",62deb2e979f22232e1aff02b,[børs]
4,1,0,0,0,0,"[""WEST"",""Nyheter"",""Shipping"",""Børs""]",62dfb08d79f22232e1d674e5,[børs]


In [25]:
i = 0
for index, row in dataset.iterrows():
  cats = []
  if row['råvaremarked']:
    cats.append('råvaremarked')
  if row['aksjer']:
    cats.append('aksjer')
  if row['børs']:
    cats.append('børs')
  if row['valuta']:
    cats.append('valuta')
  if row['other']:
    cats.append('other')

  # if 'other' in cats:
  #   cats = ['other']
    
  if cats == row['category']:
    i+=1
  else:
    print(row['article_tags'])
    print('prediction', cats)
    print('rule', row['category'])
    print(row['_id'])
print(i)

["Teknologi","metaverse","Nyheter"]
prediction ['råvaremarked', 'other']
rule ['other']
62daa47b79f22232e10b1497
["porsche","vw","Bil","Nyheter","Børs","børsnotering","tesla"]
prediction ['other']
rule ['børs']
62deb2e979f22232e1aff02b
["WEST","Nyheter","Shipping","Børs"]
prediction ['råvaremarked']
rule ['børs']
62dfb08d79f22232e1d674e5
["eqnr","equinor","kvartalsrapport","breaking","Nyheter","Energi","Børs"]
prediction ['other']
rule ['børs']
62e0d72a79f22232e1054b03
["greensill capital","Finans","hedgefond","Nyheter","Børs","credit suisse","archegos capital"]
prediction ['other']
rule ['børs']
62e100bf79f22232e10c2996
["marginskvis","viacomcbs","discovery","wall street","hedgefond","oanda","Finans","goldman sachs","baidu","Nyheter","Børs","nomura","resultatvarsel","credit suisse","archegos capital"]
prediction ['råvaremarked']
rule ['børs']
62e100de79f22232e10c2e23
["s&p","Børsoppdatering","dow jones","wall street","Nyheter","nasdaq","Børs","s&p500"]
prediction ['other']
rule ['børs

In [31]:
i = 0
for index, row in dataset.iterrows():
  cats = []
  if row['råvaremarked']:
    cats.append('råvaremarked')
  if row['aksjer']:
    cats.append('aksjer')
  if row['børs']:
    cats.append('børs')
  if row['valuta']:
    cats.append('valuta')
  if row['other']:
    cats.append('other')

  if 'other' in cats:
    cats = ['other']
    
  if cats == row['category']:
    i+=1
  else:
    print(row['article_tags'])
    print('prediction', cats)
    print('rule', row['category'])
    print(row['_id'])
print(i)

["porsche","vw","Bil","Nyheter","Børs","børsnotering","tesla"]
prediction ['other']
rule ['børs']
62deb2e979f22232e1aff02b
["WEST","Nyheter","Shipping","Børs"]
prediction ['other']
rule ['børs']
62dfb08d79f22232e1d674e5
["eqnr","equinor","kvartalsrapport","breaking","Nyheter","Energi","Børs"]
prediction ['other']
rule ['børs']
62e0d72a79f22232e1054b03
["marginskvis","viacomcbs","discovery","wall street","hedgefond","oanda","Finans","goldman sachs","baidu","Nyheter","Børs","nomura","resultatvarsel","credit suisse","archegos capital"]
prediction ['other']
rule ['børs']
62e100de79f22232e10c2e23
["OTEC","utbytte","Nyheter","Teknologi","Børs"]
prediction ['other']
rule ['børs']
62e3dff779f22232e1829095
["NORSE","flyr","lasse sandaker-nielsen","Nyheter","Luftfart"]
prediction ['børs']
rule ['other']
62e7a80b79f22232e12852c5
["opec+","Nyheter","Olje"]
prediction ['børs']
rule ['råvaremarked']
62ea8fc1148b7abcd5e9ed6f
["AMTD Digital","wallstreetbets","reddit","Nyheter","Børs"]
prediction ['oth

In [32]:
552/1408

0.39204545454545453